In [34]:
import obspy
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
import time; import datetime
from obspy.core.utcdatetime import UTCDateTime
from obspy.clients.fdsn.client import Client
from collections import defaultdict
from obspy import read_inventory
import pyarrow

In [35]:
ml_files = glob.glob('catalog_css/**/XO_*.quakeml')
print(ml_files)

['catalog_css/2018_12/XO_2018_12.quakeml', 'catalog_css/2019_06/XO_2019_06.quakeml', 'catalog_css/2019_01/XO_2019_01.quakeml', 'catalog_css/2019_07/XO_2019_07.quakeml', 'catalog_css/2018_07/XO_2018_07.quakeml', 'catalog_css/2018_09/XO_2018_09.quakeml', 'catalog_css/2018_08/XO_2018_08.quakeml', 'catalog_css/2018_06/XO_2018_06.quakeml', 'catalog_css/2018_11/XO_2018_11.quakeml', 'catalog_css/2018_10/XO_2018_10.quakeml', 'catalog_css/2019_02/XO_2019_02.quakeml', 'catalog_css/2019_05/XO_2019_05.quakeml', 'catalog_css/2019_04/XO_2019_04.quakeml', 'catalog_css/2019_03/XO_2019_03.quakeml', 'catalog_css/2018_05/XO_2018_05.quakeml']


In [36]:
count = 0
cat = obspy.core.event.Catalog()
for file in ml_files:
    count += 1
    print(count)
    cat.extend(obspy.core.event.read_events(file))

1
2
3
4
5
6
7
8


/opt/anaconda3/envs/alaska-ml/lib/python3.9/site-packages/obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2018-06-18T21:56:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
  warnings.warn(msg % (text, convert_to))


9


/opt/anaconda3/envs/alaska-ml/lib/python3.9/site-packages/obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2018-11-14T00:42:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
  warnings.warn(msg % (text, convert_to))


10


...

11


/opt/anaconda3/envs/alaska-ml/lib/python3.9/site-packages/obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2021-02-09T02:50:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
  warnings.warn(msg % (text, convert_to))


12
13


/opt/anaconda3/envs/alaska-ml/lib/python3.9/site-packages/obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2021-05-11T20:56:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
  warnings.warn(msg % (text, convert_to))


14
15


In [40]:
origins = [p.origins[0] for p in cat.events]
magnitudes = [p.magnitudes[0] for p in cat.events]


times = [p.time for p in origins]
lats = [p.latitude for p in origins]
lons = [p.longitude for p in origins]
depths = [p.depth for p in origins]
arrivals = [len(p.arrivals) for p in origins]
origin_ids = [str(p.resource_id) for p in origins]
mags = [p.mag for p in magnitudes]
mag_type = [p.magnitude_type for p in magnitudes]
origin_dict = {'time':times,'latitude':lats,'longitude':lons,'depth':depths,'num_arrivals':arrivals,'origin_id':origin_ids,'magnitude':mags,'mag_type':mag_type}

events = pd.DataFrame.from_dict(origin_dict)
events['time'] = [str(t) for t in events['time']]
events.head()

,time,latitude,longitude,depth,num_arrivals,origin_id
0,2018-12-01T13:45:05.035000Z,57.5320,-155.3753,63400.0,52,quakeml:earthquake.alaska.edu/origin/018fe91jwu
1,2018-12-02T12:45:55.588000Z,57.0288,-157.9051,400.0,49,quakeml:earthquake.alaska.edu/origin/018ffvzkpa
2,2018-12-02T14:01:26.889000Z,56.2065,-149.0909,8500.0,196,quakeml:earthquake.alaska.edu/origin/018ffwwy9e
3,2018-12-02T21:36:00.215000Z,54.9143,-161.0360,58800.0,39,quakeml:earthquake.alaska.edu/origin/018fg1afgt
4,2018-12-02T19:18:54.664000Z,56.5801,-154.9977,16800.0,39,quakeml:earthquake.alaska.edu/origin/018ffzztgm


In [41]:
events.to_parquet(path='alaska_events.parquet',version='2.6')